<a href="https://colab.research.google.com/github/roshanappa/Deep-Learning-Models./blob/master/Ann_With_Dropout_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data PreProcessing.

In [0]:
#data preprocessing.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard

In [0]:
import pickle
import time

In [0]:
NAME = 'Simple ANN Model {}'.format(int(time.time()))

In [0]:
data = pd.read_csv('/content/Churn_Modelling.csv')

In [0]:
pd.pandas.set_option('display.max_columns',None)

In [0]:
data.head()

In [0]:
data.drop(columns=['RowNumber','CustomerId'],inplace=True)

In [0]:
data.tail()

In [0]:
data.columns

In [0]:
data.isnull().sum()

In [0]:
data.drop(columns=['Surname'],inplace=True)

In [0]:
dataframe = pd.DataFrame({'Dtypes':data.dtypes,'Nuique':data.nunique(),'Null_values':data.isnull().sum(),'Duplicated':data.duplicated().sum()})

In [0]:
dataframe

In [0]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [0]:
data['Geography'].value_counts()

In [0]:
data['Geography'].value_counts().plot(kind='bar')
plt.show()

In [0]:
d = {'France':0,'Germany':1,'Spain':3}
data['Geography']=data['Geography'].map(d)

In [0]:
data['Gender'].value_counts().plot(kind='bar')
plt.show()

In [0]:
z = {'Male':0,'Female':1}
data['Gender']=data['Gender'].map(z)

## Prepare for the model.

In [0]:
data.head()

In [0]:
feature = data.columns
feature

In [0]:
x = data.drop(columns=['Exited'])
y = data['Exited']

In [0]:
x.shape,y.shape

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.80,random_state=42,stratify=y)

In [0]:
x_train.shape,x_test.shape

In [0]:
y_train.value_counts()

In [0]:
scaler = StandardScaler()
X_train  = scaler.fit_transform(x_train)
X_test   = scaler.transform(x_test)

In [0]:
from imblearn.over_sampling import SMOTE

In [0]:
sm = SMOTE(sampling_strategy=1,random_state=42,k_neighbors=5)

In [0]:
x_train_res,y_train_res = sm.fit_sample(X_train,y_train)

In [0]:

print('0 with y_train_res :-',len(y_train_res[y_train_res==0]))
print('1 with y_train_res :-',len(y_train_res[y_train_res==1]))

In [0]:
x_train_res.shape

## Artificial Neural Network With Dropout.
- During the training of the model the artificial neural layers are gets distributed in more layers.That layers trying to learn the each parameter correlation.The artificial neural network layers found some independent correlation of the data neurons work more indepently by using the **DROPOUT.**
- That prevent the neurons learning too much and by doing so we can marginally reduce overfitting of our model.

In [0]:
from tensorflow.keras.layers import Dropout

In [0]:
model = Sequential()
model.add(Dense(120,activation='relu',kernel_initializer='uniform',input_shape=[x_train_res.shape[1]]))
model.add(Dropout(0.20))
model.add(Dense(60,activation='relu',kernel_initializer='uniform'))
model.add(Dropout(0.20))
model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history = model.fit(x_train_res,y_train_res,batch_size=10,epochs=20,verbose=1,validation_split=(0.20))

In [0]:
loss_val,val_accuarcy=model.evaluate(X_test,y_test,batch_size=10,verbose=1)

In [0]:
y_pred = model.predict_classes(X_test)
y_pred

In [0]:
y_prediction = pd.Series(y_pred.ravel())
y_prediction

In [0]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [0]:
print('Accuarcy Score :',accuracy_score(y_test,y_pred))

In [0]:
mat = confusion_matrix(y_test,y_pred)
mat

In [0]:
sns.heatmap(mat,annot=True,fmt='d')
plt.show()

## Cross Validation 

In [0]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [0]:
def set_model():
  model = Sequential()
  model.add(Dense(120,activation='relu',kernel_initializer='uniform',input_shape=[x_train_res.shape[1]]))
  model.add(Dropout(0.20))
  model.add(Dense(60,activation='relu',kernel_initializer='uniform'))
  model.add(Dropout(0.20))
  model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

model

model = KerasClassifier(set_model,batch_size=10,epochs=20)
accuracies = cross_val_score(model,x_train_res,y_train_res,cv=10,n_jobs=-1)

In [0]:
accuracies

In [0]:
print('mean of accuarcies :-',accuracies.mean())
print('Standard Deviation of accuarcies :-',accuracies.std())

## Thank You.